# Using Function Call of ZhipuAI GLM API

**This tutorial is available in English and is attached below the Chinese explanation**

本代码旨在使用 GLM-4 完成简单的 Function Call 代码，通过一个查询天气的工具，让开发者清楚大模型是如何完成工具调用的任务的。

This cookbook aims to use GLM-4 to complete a simple Function Call code. Through a tool for querying weather, it allows developers to understand how the large model completes the task of tool calling.

首先，设定好调用模型的API key

First, set the API key for calling the model

In [1]:
import os
from zhipuai import ZhipuAI

os.environ["ZHIPUAI_API_KEY"] = "your api key"
client = ZhipuAI()

首先，定义一个简单的工具，这个工具不一定要真实存在，但是你需要包含以下的内容：

1. 工具的名字`name` ：这让模型返回调用工具的名称
2. 模型的描述`description`: 对工具的描述
3. 模型传入的参数`parameters`: 这里记录了工具参数的属性和内容。
4. `required`: 通常配合`parameters`，指的是需要传入模型的参数的名称，在这里，只有`location`是需要传入的。

这样就定义好了一个简单的的工具，用于查询地区的天气。

First, define a simple tool. This tool does not necessarily need to exist, but you need to include the following:

1. The name of the tool `name`: This allows the model to return the name of the calling tool
2. Model description `description`: description of the tool
3. Parameters passed in by the model: The attributes and contents of the tool parameters are recorded here.
4. `required`: usually used with `parameters`, refers to the name of the parameters that need to be passed into the model. Here, only `location` needs to be passed in.

This way defined a simple tool for querying the weather in a region.



In [2]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "获取指定城市的天气信息",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "城市，如：北京",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["c", "f"]
                    },
                },
                "required": ["location"],
            },
        }
    }
]

接着，我们就能将工具和信息组合成一个请求，并传入模型中获得回答，这里我们按照[官方教程](https://open.bigmodel.cn/dev/api#sse_example)的方式进行请求。

Then, we can combine the tools and information into a request and pass it into the model to get the answer. Here we make the request according to the [official tutorial](https://open.bigmodel.cn/dev/api#sse_example) .

In [3]:
messages = [{"role": "user", "content": "今天北京天气如何？"}]
completion = client.chat.completions.create(
    model="glm-4",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)
completion.choices

[CompletionChoice(index=0, finish_reason='tool_calls', message=CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_8434493403450401873', function=Function(arguments='{"location":"北京","unit":"c"}', name='get_current_weather'), type='function')]))]

模型返回的内容中，有效的部分其实是关于 工具调用传参的部分，观察到模型传入的参数为
```
{   
    "location":"北京",
    "unit":"celsius"
}
```
调用工具的名称为 `get_current_weather`

但是，这还不能让我们直接获取结果，因此，还需要通过工程化的方式来真正的调用这个模型，并执行这个调用请求，返回给大模型。

The valid part of the content returned by the model is actually the parameter part passed by the tool call. It can be seen that the parameters passed in the model are
````
{
    "location":"Beijing",
    "Unit": "Celsius"
}
````
The name of the calling tool is `get_current_weather`

However, this does not allow us to obtain the results directly. Therefore, we need to actually call the model through the project, execute the call request, and return it to the large model.

为了能够顺利展示工具的执行，我先对 `get_current_weather` 的代码用python进行简单实现。我使用 [心知天气](https://www.seniverse.com/) 的API并简单书写了一个请求及时天气的脚本。你需要使用 API 私钥 来填写到  `xinzhi_api_key` 中。

In order to be able to smoothly demonstrate the execution of the tool, I first simply implement the code of `get_current_weather` in python. I used the API of [Xinzhiwei](https://www.seniverse.com/) and simply wrote a script to request real-time weather. You need to use the API private key to fill in `xinzhi_api_key`.

In [4]:
import requests

xinzhi_api_key = "your xinzhi api key"


def get_current_weather(location: str, unit: str):
    url = f"https://api.seniverse.com/v3/weather/now.json?key={xinzhi_api_key}&location={location}&language=zh-Hans&unit={unit}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        weather = {
            "temperature": data["results"][0]["now"]["temperature"],
            "description": data["results"][0]["now"]["text"],
        }
        return weather
    else:
        raise Exception(
            f"Failed to retrieve weather: {response.status_code}")


/Users/zr/Code/glm-cookbook/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


接着，通过简单的 Python 脚本将模型的工具调用请求执行并返回模型相关结果。

Next, a simple Python script is used to execute the model's tool call request and return model-related results.

In [5]:
import json


def extract_function_and_execute(llm_output, messages):
    name = llm_output.choices[0].message.tool_calls[0].function.name
    params = json.loads(llm_output.choices[0].message.tool_calls[0].function.arguments)
    function_to_call = globals().get(name)
    if not function_to_call:
        raise ValueError(f"Function '{name}' not found")

    messages.append(
        {
            "role": "tool",
            "content": str(function_to_call(**params))
        }
    )
    return messages


messages = [{"role": "user", "content": "今天北京天气如何？"}]

completion = client.chat.completions.create(
    model="glm-4",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)
extract_function_and_execute(llm_output=completion, messages=messages)

final_answer = client.chat.completions.create(
    model="glm-4",
    messages=messages,
    tool_choice="auto",
)
final_answer

Completion(model='glm-4', created=1709291639, choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='今天北京的天气是晴天，温度为3摄氏度。请注意保暖，因为气温较低。', role='assistant', tool_calls=None))], request_id='8434493437810171341', id='8434493437810171341', usage=CompletionUsage(prompt_tokens=24, completion_tokens=21, total_tokens=45))

在这段代码中，工具被执行后又以 `tool` 的身份传入到大模型中，大模型会根据工具的返回给予用户回答。因此，工具调用的基本顺序是：

用户请求（和工具） -> 模型输出调用工具的信息和参数 -> 工具执行 -> 结果返回模型 -> 模型总结并回答用户

In this code, after the tool is executed, it is passed to the large model in the character of `tool`, and the large model will give the user an answer based on the return of the tool. Therefore, the basic sequence of tool calls is:

User request (and tool) -> model outputs information and parameters calling the tool -> tool execution -> results returned to model -> model summarizes and answers to user

# Close Web Search of ZhipuAI GLM API

GLM-4 API 提供了一个简单的工具，用于搜索网络上的信息。这个工具的调用方式和上面的 Function Call 类似，但是需要传入的参数和返回的结果是不同的。

由于在默认调用的时候，会默认开启网络搜索，有两种方式更比较灵活的控制网络搜索的开启和关闭。

1. 调用了其他工具，如果调用其他工具或者 Retrival ，则联网功能会被关闭
2. 通过 `enable` 参数来控制网络搜索的开启和关闭，默认开启，在本demo中提供了一种关闭的办法

**注意：目前GLM-4 API 搜索能力并不是很强大，在很多场景中优化不佳，因此在实际使用中，建议使用专业的搜索引擎API**


The GLM-4 API provides a simple tool for searching information on the web. The calling method of this tool is similar to the function call above, but the parameters that need to be determined and the returned results are different.

Since network search will be enabled by default when called by default, there are two ways to more flexibly control the opening and closing of network search.

1. Other tools are called. If other tools or searches are called, the networking function will be turned off.
2. Use the `enable` parameter to control the opening and closing of network search. It is enabled by default. This demo provides a method of closing it.

**Note: The current GLM-4 API search capability is not very strong and is not well optimized in many scenarios. In actual use, it is recommended to use a professional search engine API**


In [6]:
tools_web_search = [
    {
        "type": "web_search",
        "web_search": {
            "enable": False,
        }
    }

]

messages = [{"role": "user", "content": "2023年100强独角兽企业有哪些？"}]

completion_without_web = client.chat.completions.create(
    model="glm-4",
    messages=messages,
    tools=tools_web_search,
)
completion_without_web

Completion(model='glm-4', created=1709291646, choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='截至我知识的更新时间点（2023年初），最新的“全球独角兽企业100强”榜单尚未公布。通常这类榜单由知名的市场研究机构、财经媒体或投资机构发布，如胡润研究院、CB Insights等，它们会根据企业的估值、增长速度、行业影响力等指标进行评选。\n\n如果您想了解最新的全球独角兽企业排名，建议关注以下几家机构的发布：\n\n1. 胡润研究院发布的“全球独角兽榜”；\n2. CB Insights的“全球独角兽企业排名”；\n3. 《财富》杂志的“全球独角兽企业100强”。\n\n一般到每年的中后期，这些机构会发布当年的最新排名。对于2022年及以前的排名，这些机构网站上通常可以找到相关的历史数据。\n\n需要注意的是，独角兽企业的名单和排名会随着市场的变化、企业估值的变动以及新投资的注入等因素发生变化，因此最新的排名可能会有所不同。如果您需要获取最新的信息，建议直接访问这些机构的官方网站或通过其它权威财经新闻渠道获取。', role='assistant', tool_calls=None))], request_id='8434489383360966726', id='8434489383360966726', usage=CompletionUsage(prompt_tokens=16, completion_tokens=217, total_tokens=233))